Measure MSE - Iris

In [1]:
import json

with open('../../../results/iris_ssnp/MSE_results_ssnp.json', 'r') as file:
    MSE_results = json.load(file)

In [4]:
mean_mses = [value['mse_mean'] for value in MSE_results.values()]
overall_mean_mse = np.mean(mean_mses)
print(overall_mean_mse)

0.012988828957273211
